In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import json
pd.options.display.max_columns = None

In [2]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# Importing pickle file

In [3]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [4]:
tracks.head()

,url,artist_clean,track_clean
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [5]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [6]:
load_dotenv()

True

In [7]:
id = os.getenv("id")
secret = os.getenv("secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [8]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'alan walker'
track = 'alone'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

spotify:track:3LlmKSHR3Rs0Y3KHQLAYDk


In [9]:
for index, row in tracks.iterrows():
    print(row['artist'],'---', row['track'])
    break

migos --- bad and boujee (feat. lil uzi vert)


In [10]:
sp.search(q=q, limit=1, type='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3Aalan+walker+track%3A+alone&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7vk5e3vY1uw9plTHJAMwjN'},
       'href': 'https://api.spotify.com/v1/artists/7vk5e3vY1uw9plTHJAMwjN',
       'id': '7vk5e3vY1uw9plTHJAMwjN',
       'name': 'Alan Walker',
       'type': 'artist',
       'uri': 'spotify:artist:7vk5e3vY1uw9plTHJAMwjN'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
     

In [12]:
tqdm.pandas()

uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'].split(',')[0], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

100%|██████████| 14659/14659 [23:58<00:00, 10.19it/s] 


Voy a comprobar que la lista donde me he appendeado todo tiene la misma longitud que mi dataframe, para poder añadir los URIs como una columna

In [13]:
uris[:3]

['spotify:track:0M9ydKzuF3oZTfYYPfaGX1',
 'spotify:track:343YBumqHu19cGoGARUTsd',
 'spotify:track:7MXVkk9YMctZqd1Srtv4MB']

In [14]:
print(len(uris), tracks.shape)

14659 (14659, 3)


In [15]:
tracks['URI'] = uris
tracks.head()

,url,artist,track,URI
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,spotify:track:343YBumqHu19cGoGARUTsd
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy,spotify:track:7MXVkk9YMctZqd1Srtv4MB
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles,spotify:track:6fujklziTHa8uoM5OQSfIo


Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [16]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

[{'danceability': 0.644,
  'energy': 0.728,
  'key': 8,
  'loudness': -5.75,
  'mode': 1,
  'speechiness': 0.0351,
  'acousticness': 0.0543,
  'instrumentalness': 0.169,
  'liveness': 0.161,
  'valence': 0.43,
  'tempo': 88.993,
  'type': 'audio_features',
  'id': '3OHfY25tqY28d16oZczHc8',
  'uri': 'spotify:track:3OHfY25tqY28d16oZczHc8',
  'track_href': 'https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8',
  'duration_ms': 153947,
  'time_signature': 4}]

In [17]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

list

In [19]:
datos_uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    uri = sp.audio_features(row['URI'])
    datos_uris.append(uri)

100%|██████████| 14659/14659 [23:43<00:00, 10.29it/s] 


Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [20]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

dict

In [21]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

['danceability', 'energy', 'key', 'loudness', 'mode']

In [22]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

<ipython-input-22-a1ab41f91eb4>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
<ipython-input-22-a1ab41f91eb4>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
<ipython-input-22-a1ab41f91eb4>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
<ipython-input-22-a1ab41f91eb4>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_uris[columnas] = df_uris[0].apply(pd.Series)
<ipython-input-2

,0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"{'danceability': 0.926, 'energy': 0.666, 'key'...",0.926,0.666,11.0,-5.314,1.0,0.2440,0.0611,0.000000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
1,"{'danceability': 0.928, 'energy': 0.481, 'key'...",0.928,0.481,9.0,-9.350,0.0,0.2870,0.1050,0.000000,0.176,0.613,134.007,audio_features,343YBumqHu19cGoGARUTsd,spotify:track:343YBumqHu19cGoGARUTsd,https://api.spotify.com/v1/tracks/343YBumqHu19...,https://api.spotify.com/v1/audio-analysis/343Y...,210937.0,4.0
2,"{'danceability': 0.679, 'energy': 0.587, 'key'...",0.679,0.587,7.0,-7.015,1.0,0.2760,0.1410,0.000006,0.137,0.486,186.003,audio_features,7MXVkk9YMctZqd1Srtv4MB,spotify:track:7MXVkk9YMctZqd1Srtv4MB,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,https://api.spotify.com/v1/audio-analysis/7MXV...,230453.0,4.0
3,"{'danceability': 0.748, 'energy': 0.524, 'key'...",0.748,0.524,8.0,-5.599,1.0,0.0338,0.4140,0.000000,0.111,0.661,95.010,audio_features,7BKLCZ1jbUBVqRi2FVlTVw,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,https://api.spotify.com/v1/audio-analysis/7BKL...,244960.0,4.0
4,"{'danceability': 0.794, 'energy': 0.632, 'key'...",0.794,0.632,0.0,-6.163,1.0,0.0649,0.1420,0.000000,0.128,0.355,145.926,audio_features,6fujklziTHa8uoM5OQSfIo,spotify:track:6fujklziTHa8uoM5OQSfIo,https://api.spotify.com/v1/tracks/6fujklziTHa8...,https://api.spotify.com/v1/audio-analysis/6fuj...,291893.0,4.0


Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [23]:
tracks = tracks.merge(df_uris, left_on = 'URI', right_on='uri', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

both          19429
left_only         0
right_only        0
Name: _merge, dtype: int64

In [24]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
1,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
2,https://open.spotify.com/track/1FvU97lrWOG2NRx...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
3,https://open.spotify.com/track/1FvU97lrWOG2NRx...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.0,-5.314,1.0,0.244,0.0611,0.0,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.0,4.0
4,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love,spotify:track:343YBumqHu19cGoGARUTsd,0.928,0.481,9.0,-9.350,0.0,0.287,0.1050,0.0,0.176,0.613,134.007,audio_features,343YBumqHu19cGoGARUTsd,https://api.spotify.com/v1/tracks/343YBumqHu19...,https://api.spotify.com/v1/audio-analysis/343Y...,210937.0,4.0


# Export

In [ ]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)